## Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

## Обзор данных

#### Чтение датасета

In [2]:
df = pd.read_csv("dataset.csv")

In [3]:
df.head(2)

,Unnamed: 0,Film,Oscar Year,Film Studio/Producer(s),Award,Year of Release,Movie Time,Movie Genre,IMDB Rating,IMDB Votes,...,Tomatometer Status,Tomatometer Rating,Tomatometer Count,Audience Status,Audience Rating,Audience Count,Tomatometer Top Critics Count,Tomatometer Fresh Critics Count,Tomatometer Rotten Critics Count,Film ID
0,0,Wings,1927/28,Famous Players-Lasky,Winner,1927,144,"Drama,Romance,War",7.5,"12,221",...,Certified-Fresh,93.0,46.0,Upright,78.0,3530.0,9.0,43.0,3.0,2becf7d5-a3de-46ab-ae45-abdd6b588067
1,1,7th Heaven,1927/28,Fox,Nominee,1927,110,"Drama,Romance",7.7,"3,439",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19ed3295-a878-4fd2-8e60-5cd7b5f93dad


#### Датасет содержит следующие поля

- `Unnamed: 0` - колонка с номером фильма
- `Film` - название фильма
- `Oscar Year` - год получения премии "Оскар"
- `Film Studio/Producer(s)` - студия или продюссерский состав
- `Award` - номинация
- `Year of Release` - год выхода
- `Movie Time` - длительность фильма
- `Movie Genre` - жанр фильма
- `IMDB Rating` - рейтинг на сервисе IMDB
- `IMDB Votes` - да честно говоря, хуй знает что это
- `Movie Info` - описание фильма
- `Genres` - какого-то хуя опять жанр фильма
- `Critic Consensus` - критика
- `Content Rating` - рейтинг контента
- `Directors` - режиссёры
- `Authors` - авторы сценария
- `Actors` - актёрский состав
- `Original Release Date` - официальная дата выхода фильма
- `Streaming Release Date` - дата выхода на стриминговых платформах
- `Production Company` - площадка выхода
- `Tomatometer Status` - какие нахуй томатометры?
- `Tomatometer Rating` - пиздец, у этого ещё и рейтинг есть
- `Tomatometer Count` - ещё и счётчик
- `Audience Status` - мнение аудитории
- `Audience Rating` - рейтинг аудитории
- `Audience Count` - я бля не ебу что это
- `Tomatometer Top Critics Count` - пиздец
- `Tomatometer Fresh Critics Count` - идите нахуй с такими колонками
- `Tomatometer Rotten Critics Count` - я ебал
- `Film ID` - идентификатор фильма

#### Получение информации о данных в столбцах датафрейма

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571 entries, 0 to 570
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        571 non-null    int64  
 1   Film                              571 non-null    object 
 2   Oscar Year                        571 non-null    object 
 3   Film Studio/Producer(s)           571 non-null    object 
 4   Award                             571 non-null    object 
 5   Year of Release                   571 non-null    int64  
 6   Movie Time                        571 non-null    int64  
 7   Movie Genre                       571 non-null    object 
 8   IMDB Rating                       571 non-null    float64
 9   IMDB Votes                        571 non-null    object 
 10  Movie Info                        438 non-null    object 
 11  Genres                            439 non-null    object 
 12  Critic C

#### Выводы

- Заголовки нужно привести к lower case и заменить разделяющие пробелы на знаки нижнего подчёркивания
- Столбец `Unnamed: 0` требуется удалить
- Столбцы `Movie Genre` и `Genres` нужно объединить
- Столбец `Critic Consensus` имеет много пропущенных значений
- Данные в столбцах`Year of Release`, `Original Release Date`, `Streaming Release Date` нужно привести к типу даты
- Требуется обработка дубликатов

## Предварительная обработка данных

#### Форматирование заголовков датафрейма

In [5]:
df.columns = [column.strip().lower().replace(" ", "_").replace("/", "_or_").replace("(s)", "") for column in df.columns]

In [6]:
df.columns

Index(['unnamed:_0', 'film', 'oscar_year', 'film_studio_or_producer', 'award',
       'year_of_release', 'movie_time', 'movie_genre', 'imdb_rating',
       'imdb_votes', 'movie_info', 'genres', 'critic_consensus',
       'content_rating', 'directors', 'authors', 'actors',
       'original_release_date', 'streaming_release_date', 'production_company',
       'tomatometer_status', 'tomatometer_rating', 'tomatometer_count',
       'audience_status', 'audience_rating', 'audience_count',
       'tomatometer_top_critics_count', 'tomatometer_fresh_critics_count',
       'tomatometer_rotten_critics_count', 'film_id'],
      dtype='object')

#### Удаление первого столбца из датафрейма

In [7]:
df = df.drop(columns=df.columns[0])

#### Объединение столбцов `movie_genre` и `genres`

In [8]:
df["genre"] = df["genres"] + " " + df["movie_genre"]

In [9]:
df.drop("genres", axis=1, inplace=True)
df.drop("movie_genre", axis=1, inplace=True)

In [10]:
df["genre"]

0         Action & Adventure, Classics Drama,Romance,War
1                                                    NaN
2                                                    NaN
3      Musical & Performing Arts, Romance Drama,Music...
4                                                    NaN
                             ...                        
566                                                  NaN
567                                                  NaN
568                                                  NaN
569                                                  NaN
570                                                  NaN
Name: genre, Length: 571, dtype: object

#### Обработка столбцов с датами

Так как данные о годах получения премии "Оскар" не являются чем-то важным, можно оставить только один год

In [11]:
df["oscar_year"] = df["oscar_year"].apply(lambda year: year[:year.find("/")] if year.find("/") != -1 else year)

In [12]:
df["oscar_year"] = df["oscar_year"].apply(lambda year: str(year) + "-01-01")

In [13]:
df["oscar_year"] = pd.to_datetime(df["oscar_year"], format="%Y-%m-%d")
df["original_release_date"] = pd.to_datetime(df["oscar_year"], format="%Y-%m-%d")
df["streaming_release_date"] = pd.to_datetime(df["oscar_year"], format="%Y-%m-%d")

#### Обработка дубликатов

In [14]:
df.duplicated().sum()

0

Дубликатов нет, поэтому обрабатывать их не нужно.